Sử dụng thuật toán di truyền để tìm trọng số cho mạng nơ ron tế bào cho bài toán tìm biên ảnh.
Một số lưu ý trong quá trình code:
- Do hàm giải phương trình vi phân (ODE) chỉ nhận mảng 1 chiều (array) vào giá trị ban đầu cũng như đầu ra của x nên phải biến đổi từ dạng ma trận về mảng 1 chiều ở một số phép tính.
- Thứ tự của m, n (kích thước của ma trận đầu vào) đôi khi bị ngược nhau trong quá trình reshape. Lưu ý khi đưa vào ma trận không vuông.
- Các số dùng trong phép tính nên để ở dạng float

In [29]:
from scipy.signal import convolve2d
from scipy.integrate import ode
from PIL import Image as img
import numpy as np
import numpy.matlib as mat

In [39]:
# khai báo hàm trạng thái
def hamTrangThai(t, x, u, B, A, I, n, m):
    x = x.reshape(n, m)
    dx = -x + convolve2d(u, B, 'same') + convolve2d(hamDauRa(x), A, 'same') + I
    return dx.reshape(n*m)

# khai báo hàm đầu ra
def hamDauRa(x):
    return 0.5 * (abs(x + 1) - abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    arr2[0], arr2[2] = arr2[2], arr2[0]#nếu không có lệnh này thì ma trận tạo ra sẽ bị ngược
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Hàm lai hai số
def lai(a,b,k=1):#k là điểm lai (mặc định là 1)  67|8
    return np.round(a/10**k)*10**k+b%10**k

# Tiến hành tạo bộ trọng số mới bằng phép lai
def laiTrongSo(A,B,I):
    L=np.append((A.reshape(h*h))[:h*h//2+1],(B.reshape(h*h))[:h*h//2+1])
    L=np.append(L,I)
    L2=L.copy()

    #Hàm lai để tạo ra trọng số mới
    for i in range(len(L)):
        L2[i] = lai(L[i],L[(i+1)%len(L)])
        
    return taoMaTranDoiXung(L2[0:5]), taoMaTranDoiXung(L2[5:10]), L2[10]

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

# kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
m, n = 5, 5

# ma trận ảnh đầu vào u
u = np.array([
    [ 1, 1, 1, 1,-1],
    [ 1, 1, 1, 1,-1],
    [ 1, 1, 1, 1, 1],
    [ 1, 1, 1, 1, 1],
    [-1,-1, 1, 1, 1]], dtype=float)

# ma trận đầu ra mong muốn d
d = np.array([
    [ 1, 1, 1, 1,-1],
    [ 1,-1,-1, 1,-1],
    [ 1,-1,-1,-1, 1],
    [ 1, 1,-1,-1, 1],
    [-1,-1, 1, 1, 1]], dtype=float)

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1

# ma trận điều khiển B ban đầu
# B = np.array([
#     [-1.,-1.,-1.],
#     [-1., 8.,-1.],
#     [-1.,-1.,-1.]])
B = taoMaTranDoiXung([np.random.randint(100,999) for i in range(5)])

# ma trận phản hồi A ban đầu
# A = np.array([
#     [ 0., 0., 0.],
#     [ 0., 1., 0.],
#     [ 0., 0., 0.]])
A = taoMaTranDoiXung([np.random.randint(100,999) for i in range(5)])

# mức ngưỡng I ban đầu
I = -1.0

# sai số cho phép
E0 = 2.0

# hai giá trị ban đầu cho x với t0 = 0 và t để tính đầu ra x (t != t0)
x, t = u*0.0, 10.0 # cần đưa x0 về dạng mảng một chiều

# ma trận đầu ra ban đầu
y = hamDauRa(x)

# khởi tạo phương trình vi phân
ptvp = ode(hamTrangThai) \
.set_integrator('vode') \
.set_initial_value(x.flatten(), 0.0) \
.set_f_params(u, B, A, I, n, m)

print(A)
print(B)

[[829 689 250]
 [814 369 689]
 [250 814 829]]
[[183 345 293]
 [222 861 345]
 [293 222 183]]


In [40]:
#Tìm trọng số
E = saiSo(d,y)
loop = 100
i = 0
while(ptvp.successful() and E>E0 and i<loop):
    i+=1
    print(f'A = {A}')
    print(f'B = {B}')
    print(f'chay lan thu {i}: E = {E}')
    ptvp.set_f_params(u, B, A, I, n, m)
    x = ptvp.integrate(t)
    y = hamDauRa(x[:].reshape(n, m))
    E = saiSo(d,y)
    A, B, I = lai(A,B,I)

A = [[829 689 250]
 [814 369 689]
 [250 814 829]]
B = [[183 345 293]
 [222 861 345]
 [293 222 183]]
chay lan thu 1: E = 12.5
A = [829.1 689.1 250.1]
B = [814.1 369.1 689.1]
chay lan thu 2: E = 22.0
A = 814.1
B = nan
chay lan thu 3: E = 22.0


C:\Users\phamd\AppData\Local\Temp\ipykernel_2632\829236329.py:19: RuntimeWarning: overflow encountered in power
  return np.round(a/10**k)*10**k+b%10**k
C:\Users\phamd\AppData\Local\Temp\ipykernel_2632\829236329.py:19: RuntimeWarning: invalid value encountered in multiply
  return np.round(a/10**k)*10**k+b%10**k
C:\Users\phamd\AppData\Local\Temp\ipykernel_2632\829236329.py:19: RuntimeWarning: invalid value encountered in double_scalars
  return np.round(a/10**k)*10**k+b%10**k


TypeError: cannot unpack non-iterable numpy.float64 object

Test các hàm

In [28]:
taoMaTranDoiXung([1,2,3,4,5])
taoMaTranDoiXung([np.random.randint(100,999) for i in range(5)])

array([[911, 600, 149],
       [553, 646, 600],
       [149, 553, 911]])

In [7]:
saiSo(d,y)

4.0

In [1]:
# khởi tạo phương trình vi phân
ptvp = ode(hamTrangThai) \
.set_integrator('vode') \
.set_initial_value(x.flatten(), 0.0) \
.set_f_params(u, B, A, I, n, m)
assert ptvp.successful()
x = ptvp.integrate(t)
y = hamDauRa(x[:].reshape(n, m))
y

NameError: name 'saiSo' is not defined

In [12]:
lai(123,456,2)

156.0

In [19]:
laiTrongSo(A,B,I)

(array([[135., 668., 279.],
        [267., 652., 668.],
        [279., 267., 135.]]),
 array([[932., 453., 555.],
        [ 55., 467., 453.],
        [555.,  55., 932.]]),
 -1.0)

In [24]:
np.random.randint(100,999)

533